### Red neuronal profunda usando word2vec

In [40]:
import pandas as pd 
import sklearn as sk
import numpy as np

import re
import string
import nltk
from nltk.corpus import stopwords

from time import time
from collections import defaultdict
from tensorflow import keras

path_glove='/home/tomas/organizacion_de_datos/tp2/glove.6B.100d.txt'

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/tomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/tomas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tomas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
english_stopwords=set(stopwords.words('english'))

In [4]:
train_set=pd.read_csv("train.csv",encoding='utf-8')

In [5]:
test_set=pd.read_csv("test.csv",encoding='utf-8')

#####  Procesamiento del texto

In [6]:
def quitar_stopwords(texto):
    texto=' '.join([word for word in texto.split() if word not in english_stopwords])
    return texto

In [7]:
def quitar_menciones(texto):
    texto=' '.join([palabra for palabra in texto.split() if '@' not in palabra])
    return texto

In [8]:
Diccionario_de_lenguaje_de_internet={
"2day": "Today",
"2moro": "Tomorrow",
"2nite": "Tonight",
"4EAE": "For Ever And Ever",
"ABT":"About",
"ADN": "Any Day Now",
"AFAIC": "As Far As I’m Concerned",
"AFAICT": "As Far As I Can Tell",
"AFAIK": "As Far As I Know",
"AFAIR": "As Far As I Remember",
"AKA": "Also Known As",
"AMA": "Ask Me Anything",
"ASAIC": "As Soon As I Can",
"ASAP": "As Soon As Possible",
"ATM": "At The Moment",
"B4": "Before",
"B4N": "Bye For Now",
"Bae": "Babe/Before Anyone Else",
"BBL": "Be Back Later",
"BBT": "Be Back Tomorrow",
"BCNU": "Be Seeing You",
"BD": "Big Deal",
"BF": "Boyfriend",
"BFF": "Best Friends Forever",
"BMT": "Before My Time",
"BOL": "Be On Later",
"BOT": "Back On Topic",
"BRB": "Be Right Back",
"BRO": "Brother",
"BT": "But",
"BTW": "By The Way",
"CFY": "Calling For You",
"CU": "See You",
"CUL": "See You Later",
"Cuz": "Because",
"CYA": "Cover Your Ass",
"DAE": "Does Anyone Else",
"DBA": "Doing Business As",
"DFTBA": "Don’t Forget To Be Awesome",
"DIKU": "Do I Know You",
"DM": "Direct Message",
"DND": "Do Not Disturb",
"DR": "Double Rainbow",
"DWBH": "Don’t Worry, Be Happy",
"ELI5": "Explain Like I’m 5",
"EOM": "End Of Message",
"EOS": "End Of Story",
"F2F": "Face To Face",
"FAQ": "Frequently Asked Question",
"FB": "Facebook",
"FBF": "Flash Back Friday",
"FF": "Follow Friday",
"FIFY": "Fixed It For You",
"FITB": "Fill In The Blank",
"FML": "Fuck My Life",
"FOMO": "Fear Of Missing Out",
"FTFY": "Fixed That For You",
"FTL": "For The Loss",
"FTW": "For The Win",
"FWB": "Friends With Benefits",
"FWIW": "For What It’s Worth",
"FYE": "For Your Entertainment",
"FYEO": "For Your Eyes Only",
"FYI": "For Your Information",
"GA": "Go Ahead",
"GAL": "Get A Life",
"GF": "Girlfriend",
"GM": "Good Morning",
"GN": "Good Night",
"Gr8": "Great",
"GTR": "Getting Ready",
"HAND": "Have A Nice Day",
"HB": "Hurry Back",
"HBD": "Happy Birthday",
"HBU": "How About You",
"HMB": "Hit Be Back",
"HMU": "Hit Me Up",
"HRU": "How Are You",
"HTH": "Hope This Helps",
"IAC": "In Any Case",
"IC": "I See",
"ICYMI": "In Case You Missed It",
"IDC": "I Don’t Care",
"IDK": "I Don’t Know",
"IG": "Instagram",
"IIRC": "If I Remember Correctly",
"IKR": "I Know Right",
"ILY": "I Love You",
"IMHO": "In My Humble Opinion",
"IMMD": "It Made My Day",
"IMY": "I Miss You",
"IRL": "In Real Life",
"IS": "I’m Sorry",
"ISO": "In Search Of",
"IU2U": "It’s Up To You",
"J4F": "Just For Fun",
"JAM": "Just A Minute",
"JFY": "Just For You",
"JIC": "Just In Case",
"JK": "Just Kidding",
"JSYK": "Just So You Know",
"KK": "Okay",
"L8": "Late",
"L8R": "Later",
"LMA": "Leave Me Alone",
"LMAO": "Laughing My Ass Off",
"LMBO": "Laughing My Butt Off",
"LMK": "Let Me Know",
"LOL": "Laugh Out Loud",
"LTNS": "Long Time No See",
"LYLAS": "Love You Like A Sister",
"M/F": "Male or Female",
"M8": "Mate",
"MP": "My pleasure",
"MSM": "Mainstream Media",
"MU": "Miss You",
"MYOB": "Mind Your Own Business",
"NAGI": "Not A Good Idea",
"NBD": "No Big Deal",
"NE1": "Anyone",
"NM": "Not Much",
"NP": "No Problem",
"NSFL": "Not Safe For Life",
"NSFW": "Not Safe For Work",
"NTS": "Note To Self",
"NVM": "Never Mind",
"OC": "Original Content",
"OH": "Overheard",
"OIC": "Oh ! I See",
"OMD": "Oh My Damn",
"OMG": "Oh My Goodness",
"OMW": "On My Way",
"OT": "Off Topic",
"OFC": "Of course",
"PAW": "Parents Are Watching",
"Pls": "Please",
"POTD": "Photo Of The Day",
"POV": "Point Of View",
"PPL": "People",
"PTB": "Please Text Back",
"Q4U": "Question For You",
"QQ": "Crying",
"RBTL": "Read Between The Lines",
"RIP": "Rest In Peace",
"RL": "Real Life",
"ROFL": "Rolling On the Floor Laughing",
"RT": "Retweet",
"RTM": "Read The Manual",
"SIS": "Sister",
"SITD": "Still In The Dark",
"SM": "Social Media",
"SMH": "Shaking My Head",
"SMY": "Somebody",
"SNH": "Sarcasm Noted Here",
"SOL": "Sooner Or Later",
"Some1": "Someone",
"SRSLY": "Seriously",
"STBY": "Sucks To Be You",
"Str8": "Straight",
"SYS": "See You Soon",
"TBA": "To Be Announced",
"TBH": "To Be Honest",
"TBT": "Throwback Thursday",
"TBT": "Truth Be Told",
"TFH": "Thread From Hell",
"TFTI": "Thanks For The Invite",
"TGIF": "Thank God It’s Friday",
"THX": "Thanks",
"TIA": "Thanks in Advance",
"TIL": "Today I Learned",
"TIME": "Tears In My Eyes",
"TL;DR": "Too Long; Didn’t Read",
"TLDR":"Too long didn’t read",
"TL DR":"Too long didn’t read",
"TLC": "Tender Loving Care",
"TMI": "Too Much Information",
"TTYL": "Talk To You Later",
"TTYS": "Talk To You Soon",
"Txt": "Text",
"TYVM": "Thank You Very Much",
"U": "You",
"U4F": "You Forever",
"UR": "Your",
"VBG": "Very Big Grin",
"VSF": "Very Sad Face",
"WB": "Welcome Back",
"WBU": "What About You?",
"WEG": "Wicked Evil Grin",
"WKND": "Weekend",
"WOM": "Word of Mouth",
"WOTD": "Word Of The Day",
"Wru": "Who Are You",
"WTH": "What The Heck?",
"WTPA": "Where The Party At?",
"WU?": "What's Up",
"WU":"What's Up",
"WYCM": "Will You Call Me?",
"WYWH": "Wish You Were Here",
"XOXO": "Hugs and Kisses",
"YGM": "You’ve Got Mail",
"YNK": "You Never Know",
"YOLO": "You Only Live Once",
"YT": "YouTube",
"YW": "You’re Welcome",
}

In [9]:
def reemplazar_lenguaje_internet(texto):
    texto=texto.upper()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=Diccionario_de_lenguaje_de_internet.get(palabra,'not internet slang')
        if(traduccion!='not internet slang'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [10]:
diccionario_contracciones = {  "ain't": "is not",
                               "aint":"is not",
                               "aren't": "are not",
                               "arent":"are not",
                               "can't": "can not",
                               "cant":"can not",
                               "cause": "because",
                               "could've": "could have",
                               "couldve":"could have",
                               "couldn't": "could not",
                               "couldnt":"could not",
                               "didn't": "did not", 
                               "didnt":"did not",
                               "doesn't": "does not",
                               "doesnt":"does not",
                               "don't": "do not",
                               "dont":"do not",
                               "hadn't": "had not",
                               "hadnt":"had not",
                               "hasn't": "has not",
                               "hasnt":"has not",
                               "haven't": "have not",
                               "havent":"have not",
                               "he'd": "he would",
                               "hed":"he would",
                               "he'll": "he will",
                               "he ll":"he will",
                               "he's": "he is",
                               "hes":"he is",
                               "how'd": "how did",
                               "howd":"how did",
                               "how'd'y": "how do you",
                               "howdy":"how do you",
                               "how'll": "how will",
                               "howll":"how will",
                               "how's": "how is",
                               "hows":"how is",
                               "I'd": "I would",
                               "id":"i would",
                               "I'd've": "I would have",
                               "idve":"i would have",
                               "I'll": "I will", 
                               "Ill":"i will", #duda: si algun tweet habla de un enfermo puede traer ruido
                               "I'll've": "I will have",
                               "I'm": "I am", 
                               "im":"i am",
                               "I've": "I have",
                               "ive":"i have",
                               "i'd": "i would",
                               "i'd've": "i would have",
                               "i'll": "i will",
                               "i'll've": "i will have",
                               "i'm": "i am", 
                               "im":"i am",
                               "i've": "i have",
                               "isn't": "is not",
                               "isnt":"is not",
                               "it'd": "it would",
                               "itd":"it would",
                               "it'd've": "it would have",
                               "it'll": "it will",
                               "itll":"it will",
                               "it'll've": "it will have",
                               "it's": "it is",
                               "its":"it is",
                               "let's": "let us",
                               "lets":"let us",
                               "ma'am": "madam",
                               "maam":"madam",
                               "mayn't": "may not",
                               "maynt":"may not",
                               "might've": "might have",
                               "mightve":"might have",
                               "mightn't": "might not",
                               "mightnt":"might not",
                               "mightn't've": "might not have",
                               "must've": "must have",
                               "mustve":"must have",
                               "mustn't": "must not",
                               "mustnt":"must not",
                               "mustn't've": "must not have",
                               "needn't": "need not",
                               "neednt":"need not",
                               "needn't've": "need not have",
                               "o'clock": "of the clock",
                               "oclock":"of the clock",
                               "oughtn't": "ought not",
                               "oughtnt":"ought not",
                               "oughtn't've": "ought not have",
                               "shan't": "shall not",
                               "shant":"shall not",
                               "sha'n't": "shall not",
                               "shan't've": "shall not have",
                               "she'd": "she would",
                               "shed":"she would",
                               "she'd've": "she would have",
                               "she'll": "she will",
                               "shell":"she will",#nuevamente aca tengo ruido por dos palabras igual escritas
                               "she'll've": "she will have", 
                               "she's": "she is",
                               "shes":"she is",
                               "should've": "should have",
                               "shouldve":"should have",
                               "shouldn't": "should not",
                               "shouldnt": "should not",
                               "shouldn't've": "should not have",
                               "so've": "so have",
                               "so's": "so as",
                               "this's": "this is",
                               "that'd": "that would",
                               "that'd've": "that would have",
                               "that's": "that is",
                               "thats":"that is",
                               "there'd": "there would",
                               "thered":"there would",
                               "there'd've": "there would have",
                               "there's": "there is",
                               "theres":"there is",
                               "here's": "here is",
                               "heres":"here is",
                               "they'd": "they would",
                               "theyd":"they would",
                               "they'd've": "they would have",
                               "they'll": "they will",
                               "theyll":"they will",
                               "they'll've": "they will have", 
                               "they're": "they are",
                               "theyre":"they are",
                               "they've": "they have", 
                               "theyve":"they have",
                               "to've": "to have", 
                               "tove":"to have",
                               "wasn't": "was not",
                               "wasnt":"was not",
                               "we'd": "we would",
                               "wed":"we would",#aca nuevamente hay conflicto
                               "we'd've": "we would have",
                               "we'll": "we will",
                               "we'll've": "we will have",
                               "we're": "we are",
                               "we've": "we have",
                               "weren't": "were not",
                               "werent":"were not",
                               "what'll": "what will",
                               "whatll":"what will",
                               "what'll've": "what will have",
                               "what're": "what are", 
                               "whatre":"what are",
                               "what's": "what is",
                               "whats":"what is",
                               "what've": "what have",
                               "whatve":"what have",
                               "when's": "when is",
                               "whens":"when is",
                               "when've": "when have",
                               "whenve":"when have",
                               "where'd": "where did",
                               "whered":"where did",
                               "where's": "where is",
                               "wheres":"where is",
                               "where've": "where have",
                               "whereve":"where have",
                               "who'll": "who will", 
                               "wholl":"who will",
                               "who'll've": "who will have",
                               "who's": "who is",
                               "whos":"who is",
                               "who've": "who have",
                               "whove":"who have",
                               "why's": "why is",
                               "whys":"why is",
                               "why've": "why have",
                               "whyve":"why have",
                               "will've": "will have",
                               "willve":"will have",
                               "won't": "will not",
                               "wont":"will not",
                               "won't've": "will not have",
                               "would've": "would have",
                               "wouldve":"would have",
                               "wouldn't": "would not",
                               "wouldnt":"would not",
                               "wouldn't've": "would not have",
                               "y'all": "you all",
                               "yall":"you all",
                               "y'all'd": "you all would",
                               "yalld":"you all would",
                               "y'all'd've": "you all would have",
                               "y'all're": "you all are",
                               "yallre":"you all are",
                               "y'all've": "you all have",
                               "you'd": "you would",
                               "youd":"you would",
                               "you'd've": "you would have",
                               "you'll": "you will",
                               "youll":"you will",
                               "you'll've": "you will have",
                               "you're": "you are",
                               "youre":"you are",
                               "you've": "you have",
                               "youve":"you have"}


In [11]:
def reemplazar_contracciones(texto):
    texto=texto.lower()
    palabras=texto.split()
    palabras_procesadas=[]
    for palabra in palabras:
        traduccion=diccionario_contracciones.get(palabra,'no es contraccion')
        if(traduccion!='no es contraccion'):
            lista_aux=traduccion.split()
            for x in lista_aux:
                palabras_procesadas.append(x.lower())
        else:
            palabras_procesadas.append(palabra.lower())
    texto=' '.join([word for word in palabras_procesadas])
    return texto

In [12]:
#es necesario importar la libreria re (regular expression) para poder usar esta funcion
#ya esta bajada en el comienzo del notebook pero si lo que quieren es analizar la documentación
#lo pueden revisar en google
#falta borrar los arroba
def limpiar_texto(texto):
    texto = reemplazar_lenguaje_internet(texto)
    texto=reemplazar_contracciones(texto)
    texto=texto.lower()
    texto = re.sub('\[.*?\]', '', texto) # remove text in square brackets
    texto = re.sub('https?://\S+|www\.\S+', '', texto) # remove URLs
    texto = re.sub('<.*?>+', '', texto) # remove html tags
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto) # remove punctuation
    texto = re.sub('\n', '', texto) # remove words conatinaing numbers
    texto = re.sub('\w*\d\w*', '', texto)
    texto = re.sub('[‘’“”…]', '', texto)
    texto = quitar_stopwords(texto)
    texto = quitar_menciones(texto)

    return texto

In [16]:
train_set['text']=train_set['text'].apply(limpiar_texto)

In [17]:
from nltk import WordNetLemmatizer

In [18]:
lemmatizer=WordNetLemmatizer()

In [19]:
def lemmatizar_texto(texto):
    lista_palabras=nltk.word_tokenize(texto)
    texto=' '.join([lemmatizer.lemmatize(word) for word in lista_palabras])
    return texto
#nota: la función esta definida aca, pero para ver los resultados que produce hay
#que ir al comienzo del notebook y ejecutar la celda de lemmatizar texto luego
#de haber limpiado el texto original

In [20]:
from nltk.corpus import wordnet

def get_wordnet_pos(palabra):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([palabra])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [21]:
def lemmatizar_con_pos(texto):
    lista_palabras=texto.split()
    texto=' '.join(lemmatizer.lemmatize(palabra,get_wordnet_pos(palabra)) for palabra in lista_palabras)
    return texto

In [22]:
train_set['text']=train_set['text'].apply(lemmatizar_con_pos)

### Tokenización del texto

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [26]:
X=train_set['text'].copy().values
Y=train_set['target'].copy().values
#separamos el train set en dos, un conjunto de entrenamiento y uno de validacion, ratio: 90-10
X_para_entrenar,X_para_evaluar,Y_para_entrenar,Y_para_evaluar=train_test_split(X,Y,test_size=0.1,random_state=1000)

#se hace un análisis del target usando NLP
X_train=X_para_entrenar
Y_train=Y_para_entrenar
X_test=test_set['text'].copy().values

print("Shape de X_train: "+ str(X_train.shape))
print("Shape de X_test:" + str(X_test.shape))
todos_los_textos=np.concatenate([X_train,X_test])
print("Shape de todos_los_textos:" + str(todos_los_textos.shape))

#estan todos los textos concatenados, hay que 'entrenar' al tokenizador
objeto_tokenizador=Tokenizer()
objeto_tokenizador.fit_on_texts(todos_los_textos)

#necesito una cota de la longitud de cada palabra de los textos que 
#se van a analizar
longitud_maxima=max([len(s.split()) for s in todos_los_textos])
print("longitud maxima: " + str(longitud_maxima))

#necesito saber cuantas palabras tengo en mi 'diccionario' de palabras
tamanio_de_vocabulario=len(objeto_tokenizador.word_index) +1
print("tamanio de vocabulario: " + str(tamanio_de_vocabulario))

#ahora que tengo esto, es tiempo de tokenizar cada uno de los tweets
#y agregar el padding necesario
X_train_tokens=objeto_tokenizador.texts_to_sequences(X_train)
X_train_pad=pad_sequences(X_train_tokens,maxlen=longitud_maxima,padding='post')

X_test_tokens=objeto_tokenizador.texts_to_sequences(X_test)
X_test_pad=pad_sequences(X_test_tokens,maxlen=longitud_maxima,padding='post')

X_para_evaluar=pad_sequences(objeto_tokenizador.texts_to_sequences(X_para_evaluar),maxlen=longitud_maxima,padding='post')

Shape de X_train: (6851,)
Shape de X_test:(3263,)
Shape de todos_los_textos:(10114,)
longitud maxima: 31
tamanio de vocabulario: 27401


In [38]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [42]:
dimension_embedding=100
embedding_matrix = create_embedding_matrix(path_glove,objeto_tokenizador.word_index, dimension_embedding)


nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
print(nonzero_elements/tamanio_de_vocabulario)

0.5121710886463998


In [116]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import concatenate
from tensorflow.keras import Model
def crear_modelo(num_filtros,tamanio_kernel1,tamanio_kernel2,tamanio_kernel3,tamanio_vocab,dimension_embedding,max_len):
    
    #red neuronal de 3 canales, representando 3 distintos n-gramas
    #voy a usar n=2,3,5
    #no se va a usar glove embeddings por la escasa 
    
    #canal 1
    input1=Input(shape=(max_len,))
    embedding1=Embedding(tamanio_vocab,dimension_embedding)(input1)
    convolution1=Conv1D(num_filtros,tamanio_kernel1,activation='relu')(embedding1)
    pooling1=MaxPooling1D(pool_size=2)(convolution1)
    flat1=Flatten()(pooling1)

    #canal 2
    input2=Input(shape=(max_len,))
    embedding2=Embedding(tamanio_vocab,dimension_embedding)(input2)
    convolution2=Conv1D(num_filtros,tamanio_kernel2,activation='relu')(embedding2)
    pooling2=MaxPooling1D(pool_size=2)(convolution2)
    flat2=Flatten()(pooling2)

    #canal 3
    input3=Input(shape=(max_len,))
    embedding3=Embedding(tamanio_vocab,dimension_embedding)(input3)
    convolution3=Conv1D(num_filtros,tamanio_kernel3,activation='relu')(embedding3)
    pooling3=MaxPooling1D(pool_size=2)(convolution3)
    flat3=Flatten()(pooling3)

   #merge
    merged=concatenate([flat1,flat2,flat3])
    
    #interpretacion
    dense1=Dense(10,activation='relu')(merged)
    output=Dense(1,activation='sigmoid')(dense1)
    model=Model(inputs=[input1,input2,input3],outputs=[output])
    
    #compilar
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    #resumen
    print(model.summary())
    
    return model

In [117]:
modelo=crear_modelo(32,2,3,4,tamanio_de_vocabulario,dimension_embedding,longitud_maxima)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, 31, 100)      2740100     input_25[0][0]                   
____________________________________________________________________________________________

In [110]:
history=modelo.fit([X_train_pad, X_train_pad, X_train_pad],Y_train,epochs=15,batch_size=100)

Epoch 1/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0096 - acc: 0.9953
Epoch 2/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0101 - acc: 0.9952
Epoch 3/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0094 - acc: 0.9956
Epoch 4/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0102 - acc: 0.9953
Epoch 5/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0092 - acc: 0.9958
Epoch 6/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0096 - acc: 0.9952
Epoch 7/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0088 - acc: 0.9956
Epoch 8/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0089 - acc: 0.9955
Epoch 9/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0083 - acc: 0.9956
Epoch 10/15
6851/6851 [==============================] - 10s 1ms/sample - loss: 0.0088 - acc: 0.9955

In [111]:
loss,accuracy=modelo.evaluate([X_train_pad,X_train_pad,X_train_pad],Y_train,verbose=False)

In [112]:
print("Training Accuracy: {:.4f}".format(accuracy))

Training Accuracy: 0.9968


In [113]:
loss, accuracy =modelo.evaluate([X_para_evaluar,X_para_evaluar,X_para_evaluar], Y_para_evaluar, verbose=False)

In [114]:
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7953


In [95]:
predicciones=modelo.predict([X_test_pad,X_test_pad,X_test_pad])

In [96]:
resultado=[]
for numero in predicciones:
    if numero>=0.5:
        resultado.append(1)
    else:
        resultado.append(0)

In [97]:
sampleSubmission=pd.read_csv('sample_submission.csv')
output=pd.DataFrame({'id':sampleSubmission.id,'target':resultado})
output.to_csv('submission.csv',index=False)

In [115]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

In [118]:
#settings:
epochs=20
#longitud_maxima (calculada antes)
#embedding dimension(calculada antes)
output_file='output.txt'

modelo=KerasClassifier(build_fn=crear_modelo,epochs=epochs,batch_size=10,verbose=False)

In [124]:
param_grid=dict(num_filtros=[32,64,128],tamanio_kernel1=[3,5,7],tamanio_kernel2=[1,2,3],tamanio_kernel3=[4,6,5],tamanio_vocab=[tamanio_de_vocabulario],dimension_embedding=[dimension_embedding],max_len=[longitud_maxima])

In [125]:
grid=RandomizedSearchCV(estimator=modelo, param_distributions=param_grid,cv=4,verbose=1,n_iter=5)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Model: "model_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_91 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_92 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_93 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_90 (Embedding)        (None, 31, 100)      2740100     input_91[0][0]                   
___________________________________________________________________________________________

Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_97 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_98 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_99 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_96 (Embedding)        (None, 31, 100)      2740100     input_97[0][0]                   
___________________________________________________________________________________________

Model: "model_29"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_103 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_104 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_105 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_102 (Embedding)       (None, 31, 100)      2740100     input_103[0][0]                  
___________________________________________________________________________________________

...

Model: "model_41"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_139 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_140 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_141 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_138 (Embedding)       (None, 31, 100)      2740100     input_139[0][0]                  
___________________________________________________________________________________________

Model: "model_43"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_145 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_146 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_147 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_144 (Embedding)       (None, 31, 100)      2740100     input_145[0][0]                  
___________________________________________________________________________________________

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    9.0s finished


Model: "model_45"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_151 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_152 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_153 (InputLayer)          [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding_150 (Embedding)       (None, 31, 100)      2740100     input_151[0][0]                  
___________________________________________________________________________________________

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 1 arrays: [array([[  211,    81,   135, ...,     0,     0,     0],
       [   48,  1026,     5, ...,     0,     0,     0],
       [  372,    76,   110, ...,     0,     0,     0],
       ...,
       [   11,   31...

In [103]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import concatenate
from tensorflow.keras import Model
def crear_modelo(num_filtros,tamanio_kernel,tamanio_vocab,dimension_embedding,max_len):
    
    #red neuronal de 3 canales, representando 3 distintos n-gramas
    #voy a usar n=2,3,5
    #no se va a usar glove embeddings por la escasa 
    
    #canal 1
    input1=Input(shape=(max_len,))
    embedding1=Embedding(tamanio_vocab,dimension_embedding)(input1)
    convolution1=Conv1D(filters=32,kernel_size=2,activation='relu')(embedding1)
    pooling1=MaxPooling1D(pool_size=2)(convolution1)
    flat1=Flatten()(pooling1)

    #canal 2
    input2=Input(shape=(max_len,))
    embedding2=Embedding(tamanio_vocab,dimension_embedding)(input2)
    convolution2=Conv1D(filters=32,kernel_size=2,activation='relu')(embedding2)
    pooling2=MaxPooling1D(pool_size=2)(convolution2)
    flat2=Flatten()(pooling2)

    #canal 3
    input3=Input(shape=(max_len,))
    embedding3=Embedding(tamanio_vocab,dimension_embedding)(input3)
    convolution3=Conv1D(filters=32,kernel_size=2,activation='relu')(embedding3)
    pooling3=MaxPooling1D(pool_size=2)(convolution3)
    flat3=Flatten()(pooling3)

   #merge
    merged=concatenate([flat1,flat2,flat3])
    
    #interpretacion
    dense1=Dense(10,activation='relu')(merged)
    output=Dense(1,activation='sigmoid')(dense1)
    model=Model(inputs=[input1,input2,input3],outputs=[output])
    
    #compilar
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    #resumen
    print(model.summary())
    
    return model

In [131]:
if 'time' in english_stopwords:
    print("time esta en los stopwords")